# Explore data variance

## Setup

In [1]:
sample_file <- "sample_combined_2019-09-16.feather"

In [2]:
library(DataAnalysisTools)
library(tidyverse)
library(oildata)

── Attaching packages ────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ───────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [3]:
set.seed(6)

## Load data

In [4]:
sample <- feather::read_feather(sample_file)
sample_n(sample, 5)

YEAR,OPERATOR_ID,CRUDE_AGE_UNKNOWN_MILES,CRUDE_AVG_AGE,CRUDE_INCIDENTS,CRUDE_MILES,CRUDE_MILES_1940,CRUDE_MILES_1950,CRUDE_MILES_1960,CRUDE_MILES_1970,⋯,HVL_MILES_2000_3,HVL_MILES_2010_3,NON_HVL_MILES_1940_3,NON_HVL_MILES_1950_3,NON_HVL_MILES_1960_3,NON_HVL_MILES_1970_3,NON_HVL_MILES_1980_3,NON_HVL_MILES_1990_3,NON_HVL_MILES_2000_3,NON_HVL_MILES_2010_3
<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2015,26049,0,31.3655,1,2.2,45.8,8.8,0,27.2,⋯,0.00,0.00,0,151.30,0.20,0.00,52.70,0.20,1.40,0.60
2011,ONEOK (Group),0,0.0000,0,0.0,0.0,0.0,0,0.0,⋯,4705.96,183.28,0,443.19,326.37,116.70,71.94,29.19,0.26,0.45
2005,4805,0,0.0000,0,0.0,0.0,0.0,0,0.0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2006,25146,0,0.0000,0,0.0,0.0,0.0,0,0.0,⋯,9.10,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2011,Chevron (Group),0,0.0000,0,0.0,0.0,0.0,0,0.0,⋯,154.69,0.00,170,1753.45,673.12,97.09,57.32,0.00,38.35,0.00


In [5]:
data(pipelines_2010)
sample_n(pipelines_2010, 5)

DATAFILE_AS_OF,Year,REPORT_NUMBER,SUPPLEMENTAL_NUMBER,ID,Name,PARTA4STREET,PARTA4CITY,PARTA4STATE,PARTA4ZIP,⋯,PARTE2010HF,PARTE2010LF,PARTE2010TOTAL,PARTETOTAL,PARTETOTALHF,PARTETOTALLF,REPORT_SUBMISSION_TYPE,REPORT_DATE,FILING_DATE,FORM_REV
<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dttm>,<dttm>,<chr>
2019-11-01 07:22:30,2011,20120302,11340,26085,"PLAINS MARKETING, L.P.",333 Clay Street Suite 1600,CUSHING,TX,77002,⋯,0.00,0,0.00,6.27,6.27,0.00,INITIAL,2012-06-14 11:14:36,2012-06-14 11:14:36,7000-1.1 (Rev. 06-2011)
2019-11-01 08:46:27,2017,20183187,15699,39571,"HPIP GONZALES HOLDINGS, LLC",1415 LOUISIANA STREET SUITE 1600,HOUSTON,TX,77002,⋯,25.55,0,25.55,25.55,25.55,0.00,INITIAL,2018-06-01 11:59:49,2018-06-01 11:59:49,7000-1.1 (Rev. 06-2014)
2019-11-01 07:58:02,2016,20172986,15461,30658,KERN OIL & REFINING CO.,7724 E. PANAMA LANE,BAKERSFIELD,CA,93307,⋯,0.00,0,0.00,18.67,18.67,0.00,SUPPLEMENTAL,2017-07-21 11:44:25,2017-12-18 06:20:48,7000-1.1 (Rev. 06-2014)
2019-11-01 07:47:16,2014,20151610,13759,2731,CHEVRON PIPE LINE CO,4800 FOURNACE PLACE RM C388,BELLAIRE,TX,77401-2324,⋯,7.00,0,7.00,608.20,465.30,142.90,INITIAL,2015-06-15 14:27:08,2015-06-15 14:27:08,7000-1.1 (Rev. 06-2014)
2019-11-01 07:33:29,2013,20141041,13114,32109,"ONEOK NGL PIPELINE, LLC",100 WEST FIFTH STREET,TULSA,OK,74102,⋯,6.28,0,6.28,588.43,262.68,325.75,SUPPLEMENTAL,2014-06-15 19:46:16,2014-06-17 13:25:03,7000-1.1 (Rev. 06-2011)


## Add missing names

In [6]:
# To be used when there are IDs left in the name column
add_missing_names <- function(sample, id_col, dataset = pipelines_2010){
    id_col_type <- class(sample[[id_col]])
    
    # Prepare dataset to merge with
    all_names <- dataset %>%
        select(ID, Name, Year) %>%
        group_by(ID) %>%
        filter(Year == max(Year)) %>%
        slice(1) %>%
        select(-Year)
    all_names$ID <- as.character(all_names$ID)
    all_names <- rename(all_names, "temp_name" = "Name")
    
    # Do the merging to obtain the missing names
    sample[id_col] <- as.character(sample[[id_col]])
    sample <- left_join(sample, all_names, by = setNames("ID", id_col))
    name_missing <- !is.na(sample$temp_name)  # The name of the company can only be missing if joining by OPERATOR_ID is successfuly (bc then there is an ID, not a Name there)
    sample[name_missing, ][id_col] <- sample[name_missing, ]["temp_name"]
    
    # Cleanup
    sample <- select(sample, -temp_name)
    sample[id_col] <- as.type(sample[[id_col]], "factor")
    
    return(sample)
}

In [7]:
sample <- add_missing_names(sample, "OPERATOR_ID")
sample <- rename(sample, "Name" = "OPERATOR_ID")

sample_n(sample, 5)

YEAR,Name,CRUDE_AGE_UNKNOWN_MILES,CRUDE_AVG_AGE,CRUDE_INCIDENTS,CRUDE_MILES,CRUDE_MILES_1940,CRUDE_MILES_1950,CRUDE_MILES_1960,CRUDE_MILES_1970,⋯,HVL_MILES_2000_3,HVL_MILES_2010_3,NON_HVL_MILES_1940_3,NON_HVL_MILES_1950_3,NON_HVL_MILES_1960_3,NON_HVL_MILES_1970_3,NON_HVL_MILES_1980_3,NON_HVL_MILES_1990_3,NON_HVL_MILES_2000_3,NON_HVL_MILES_2010_3
<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2014,"FRIO LASALLE PIPELINE, LP",0,0.00000,0,0,0,0,0,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2011,"EQUISTAR CHEMICALS, L.P.",0,0.00000,0,0,0,0,0,0,⋯,21.57,0.0,0.0,99.62,82.11,234.89,25.43,137.28,0.00,0.0
2010,Dow (Group),0,0.00000,0,0,0,0,0,0,⋯,129.98,0.0,1.5,0.00,33.18,51.88,38.79,48.90,30.89,1.4
2013,TAMPA BAY PIPELINE CO.,0,0.00000,0,0,0,0,0,0,⋯,62.25,3.6,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0
2010,ExxonMobil (Group),1,28.18872,1,31,1,2,2,7,⋯,443.50,0.0,635.0,288.00,900.70,240.00,484.00,153.00,296.80,0.0


## Sample overview

### Largest operators by total miles

In [8]:
sample %>%
    group_by(Name) %>%
    filter(TOTAL_MILES == max(TOTAL_MILES)) %>%
    ungroup() %>%
    top_n(5, TOTAL_MILES) %>%
    arrange(desc(TOTAL_MILES)) %>%
    select(Name, TOTAL_MILES) %>%
    jupyter_styling(font_size = 16)

Name,TOTAL_MILES
ENTERPRISE PRODUCTS OPERATING LLC,13401.76
ONEOK (Group),10784.50
Phillips 66 (Group),10356.00
Magellan (Group),9162.00
Kinder Morgan (Group),7976.34


### Save slides

In [9]:
system("jupyter nbconvert explore_data_variance.ipynb --to slides")